## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:

[[citizens_st_mobile]]

[[flats_dir_partner]]

[[entries_installation_points_dir_partner]]

[[installation_point_st_partner]]

[[companies_st_partner]]
___

In [10]:
query_text = """--sql
    SELECT
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.citizen_id AS citizen_id,
        citizens_st_mobile.trial_available AS trial_available,
        citizens_st_mobile.state AS state,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        flats_dir_partner.address_uuid AS address_uuid,
        entries_installation_points_dir_partner.city AS city,
        installation_point_st_partner.partner_uuid AS partner_uuid,
        installation_point_st_partner.monetization AS monetization,
        installation_point_st_partner.monetization_is_allowed AS monetization_is_allowed,
        flats_dir_partner.created_at AS flat_created_at,
        citizens_st_mobile.activated_at AS activated_at
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.flats_dir_partner_ch AS flats_dir_partner 
        ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points_dir_partner 
        ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
    LEFT JOIN db1.installation_point_st_partner_ch AS installation_point_st_partner
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
    LEFT JOIN db1.companies_st_partner_ch AS companies_st_partner
        ON installation_point_st_partner.partner_uuid = companies_st_partner.partner_uuid
        AND companies_st_partner.report_date = citizens_st_mobile.report_date
    WHERE citizens_st_mobile.state = 'activated'
    limit 10
    """

ch.get_schema(query_text)

(
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String,
    `city` String,
    `partner_uuid` String,
    `monetization` Int16,
    `monetization_is_allowed` Int16,
    `flat_created_at` DateTime,
    `activated_at` DateTime
)


In [13]:
query_text = """--sql
CREATE TABLE db1.rep_mobile_citizens_id_city_partner
(
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String,
    `city` String,
    `partner_uuid` String,
    `monetization` Int16,
    `monetization_is_allowed` Int16,
    `flat_created_at` DateTime,
    `activated_at` DateTime
)
ENGINE = MergeTree()
ORDER BY citizen_id
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [2]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.rep_mobile_citizens_id_city_partner_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR 5 MINUTE TO db1.rep_mobile_citizens_id_city_partner AS
   SELECT
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.citizen_id AS citizen_id,
        citizens_st_mobile.trial_available AS trial_available,
        citizens_st_mobile.state AS state,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        flats_dir_partner.address_uuid AS address_uuid,
        entries_installation_points_dir_partner.city AS city,
        installation_point_st_partner.partner_uuid AS partner_uuid,
        installation_point_st_partner.monetization AS monetization,
        installation_point_st_partner.monetization_is_allowed AS monetization_is_allowed,
        flats_dir_partner.`created_at` AS flat_created_at,
        citizens_st_mobile_ch.activated_at AS activated_at
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.flats_dir_partner_ch AS flats_dir_partner 
        ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points_dir_partner 
        ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
    LEFT JOIN db1.installation_point_st_partner_ch AS installation_point_st_partner
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
    LEFT JOIN db1.companies_st_partner_ch AS companies_st_partner
        ON installation_point_st_partner.partner_uuid = companies_st_partner.partner_uuid
        AND companies_st_partner.report_date = citizens_st_mobile.report_date
    WHERE citizens_st_mobile.state = 'activated'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [4]:
query_text = """--sql
    SELECT
        *
    FROM db1.rep_mobile_citizens_id_city_partner
    ORDER BY report_date
    limit 100
    """

ch.query_run(query_text)


report_date,citizen_id,trial_available,state,flat_uuid,address_uuid,city,partner_uuid,monetization,monetization_is_allowed,flat_created_at,activated_at
date,i64,i64,str,str,str,str,str,i64,i64,datetime[ns],datetime[ns]
2023-07-10,596631,0,"""activated""","""b2771e72-4af2-46fe-92f5-ae9166…","""97fb2de5-0ade-4839-94d7-0cbb5c…","""Тольятти""","""""",0,0,2022-03-31 09:54:06,2022-09-09 04:04:24
2023-07-10,596688,0,"""activated""","""7e261bf6-4766-46a1-9dfe-bc1c78…","""c281b3ac-a589-4c27-9083-8cf14f…","""Стерлитамак""","""""",0,0,2022-04-03 15:17:06,2022-04-03 18:17:07
2023-07-10,596685,0,"""activated""","""5d6943e6-decc-4c7a-88ec-33d052…","""141d2238-c3a6-4338-9590-7fa279…","""Краснодар""","""""",0,0,2021-11-26 07:29:56,2022-04-03 18:17:28
2023-07-10,596684,0,"""activated""","""81ff0fd5-d885-4388-8fb9-ee8a27…","""c281b3ac-a589-4c27-9083-8cf14f…","""Стерлитамак""","""""",0,0,2022-04-03 15:10:13,2022-04-03 18:16:28
2023-07-10,596647,0,"""activated""","""a7a375b6-1769-410c-8bd1-2cff78…","""fe925b29-92dc-4db7-9df8-c0b57b…","""Миасс""","""""",0,0,2022-04-06 12:39:34,2022-04-06 15:39:35
…,…,…,…,…,…,…,…,…,…,…,…
2023-07-10,596797,0,"""activated""","""c0b5a06e-d07f-417e-9225-d4101e…","""976477ce-7634-49e9-a2d5-197c54…","""Кемерово""","""""",0,0,2023-05-08 05:28:33,2023-05-08 08:28:34
2023-07-10,596793,0,"""activated""","""""","""""","""""","""""",0,0,1970-01-01 00:00:00,2022-04-03 20:02:21
2023-07-10,596809,0,"""activated""","""fbf37cbd-98e4-493b-82cf-e66791…","""a8eaec9c-250b-4ca4-a990-db01ba…","""Ростов-на-Дону""","""""",0,0,2022-03-19 18:45:32,2022-04-27 14:39:22


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.rep_mobile_citizens_id_city_partner DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [11]:
query_text = """--sql
    DROP TABLE db1.rep_mobile_citizens_id_city_partner_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [12]:
query_text = """--sql
    DROP TABLE db1.rep_mobile_citizens_id_city_partner
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.rep_mobile_citizens_id_city_partner_mv
"""

ch.query_run(query_text)